In [36]:
from  urllib  import request
import logging
from pathlib import Path
import numpy as np
import pandas as pd
import re
import MeCab
from gensim import corpora, models,matutils
import random
from tqdm import tqdm_notebook as tqdm
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

In [3]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [4]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]
print(stopwords[:3])

['あそこ', 'あたり', 'あちら']


In [5]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/English.txt")
stopwords += [line.decode("utf-8").strip() for line in res]
print(stopwords[-3:])

["you've", 'z', 'zero']


In [6]:
class Tokenizer:
    def __init__(self, stopwords, parser=None, include_pos=None, exclude_posdetail=None, exclude_reg=None):
    
        self.stopwords = stopwords
        self.include_pos = include_pos if include_pos else  ["名詞", "動詞", "形容詞"]
        self.exclude_posdetail = exclude_posdetail if exclude_posdetail else ["接尾", "数"]
        self.exclude_reg = exclude_reg if exclude_reg else r"$^"  # no matching reg
        if parser:
            self.parser = parser
        else:
            mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
            self.parser = mecab.parse
            

    def tokenize(self, text, show_pos=False):
        text = re.sub(r"https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+", "", text)    #URL
        text = re.sub(r"\"?([-a-zA-Z0-9.`?{}]+\.jp)\"?" ,"", text)  # xxx.jp 
        text = text.lower()
        l = [line.split("\t") for line in self.parser(text).split("\n")]
        res = [
            i[2] if not show_pos else (i[2],i[3]) for i in l 
                if len(i) >=4 # has POS.
                    and i[3].split("-")[0] in self.include_pos
                    and i[3].split("-")[1] not in self.exclude_posdetail
                    and not re.search(r"(-|−)\d", i[2])
                    and not re.search(self.exclude_reg, i[2])
                    and i[2] not in self.stopwords          
            ]
        return res

In [7]:
t = Tokenizer(stopwords, mecab.parse, exclude_reg=r"\d(年|月|日)")

In [8]:
t.tokenize("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認める', '自分自身', '若さ故の過ち']

In [9]:
pos_doc = []
neg_doc = []

In [10]:
with open("data/amazon_ja/pos.txt") as f:
    pos_doc = [t.tokenize(doc) for doc in tqdm(f.readlines())]
print(pos_doc[:5])

HBox(children=(IntProgress(value=0, max=20887), HTML(value='')))


[['子ども', 'プール', '使う', '購入', 'する', '朝', '張る', 'プール', '水温', '上げる', 'の', '音', 'する', '前日', '膨らむ', '遊ぶ', '直前', '追加', '空気入れ', 'する', '使う', 'まくる', '空気', '入れる', '以外', '抜く', '出来る', 'プール', 'コンパクト', '収納', 'できる', '終わる', 'なる', '落ち葉', 'なる', '出す', 'ころ', '砂利', '紛れる', '落ち葉', '向かう', '使う', '落ち葉', '集め', 'なる', '思う', '落ち葉', '使う'], ['サンシェードプール', '同時', '購入', 'する', 'ミニサイズ', 'プール', '使う', 'いる', '足踏み', 'ポンプ', '電動', 'ん', '感じる', '笑'], ['ハンディー', '掃除', 'モーター', '掃除機', '吸う', '排気', '活用', 'する', '作動', 'ハンディー', '掃除機', 'そのもの', '空気', '入る', '小さい', '軽い', '安い', '空気', '入る', '買う', '良い'], ['シングルサイズエアベット', '膨らむ', '電池', '方式', 'もつ', 'いる', 'かかる', '車', 'コンセント', '付く', 'いる', 'おすすめ', '騒音', '車', '使う', '聴こえる', 'びっくり', 'する', '満足', 'する'], ['子供', 'プール', '空気入れ', '購入', '初め', '余計', 'お金', 'かける', '人力', 'できる', '思う', 'プール', '購入', 'する', 'の', 'なる', '膨らむ', '商品', '購入', 'する', '使う', 'みる', '五分', 'かかる', '膨らむ', 'できる', '子供', '水遊び', 'する', '騒ぐ', '準備', 'できる', '助かる', 'いる', '空気', '抜く', 'できる', '遊び', '終わる', '空気', '抜く', '片付ける', '数回', '遊ぶ', '我が家', '合う', 'いる', '助かる', 'い

In [11]:
with open("data/amazon_ja/neg.txt") as f:
    neg_doc = [t.tokenize(doc) for doc in tqdm(f.readlines())]
print(neg_doc[:5])

HBox(children=(IntProgress(value=0, max=5373), HTML(value='')))


[['音', '大きい', 'の', '熱', '持つ', '使える', 'ただ', '持つ', 'すぎる', '心配'], ['fieldoor', 'クイーン', 'サイズ', 'エアー', 'ベッド', '購入', 'する', '使い', '悪い', 'の', 'エアー', '注入', '本体', '置く', '入れる', 'する', '排気口', 'なる', '置く', '状態', '注入', '出来る', 'の', 'ネック'], ['子供', 'プール', '空気入れ', '空気抜き', '為', '購入', 'する', '空気入れ', '空気抜き', '性能', '問題', 'の', 'うるさい', '学校', '黒板消し', 'クリーナー', '音', '似る', '近所', '迷惑', '使う', 'にくい'], ['モーター', 'なる', '掃除機', '音', '大きい', '8人', '入れる', 'プール', '膨らます', '買う', '15分', 'かかる', 'する'], ['電動', 'エアポンプ', '使う', '比較', 'できる', '差し込み', '長い', 'すぎる', 'プール', '吸入', '収まる', '押さえる', '空気', '入る', '面倒くさい', '手動', '入れる', 'いい', 'の', '残念', '3点', 'する']]


In [12]:
d = corpora.Dictionary(pos_doc+neg_doc)

In [13]:
pos_bow = [d.doc2bow(doc) for doc  in tqdm(pos_doc)]

HBox(children=(IntProgress(value=0, max=20887), HTML(value='')))

In [14]:
neg_bow = [d.doc2bow(doc) for doc  in tqdm(neg_doc)]

HBox(children=(IntProgress(value=0, max=5373), HTML(value='')))

In [23]:
df = pd.DataFrame([len(b) for b in pos_bow+neg_bow],columns=["length"])

In [24]:
df.head()

,length
0,36
1,14
2,17
3,17
4,34


In [25]:
df.describe()

,length
count,26260.000000
mean,23.027380
std,23.030095
min,1.000000
25%,11.000000
50%,17.000000
75%,28.000000
max,803.000000


In [19]:
len(d)

33563

In [18]:
dense = list(matutils.corpus2dense(pos_bow+neg_bow,  num_terms=len(d)))

In [20]:
dense = np.array(dense)

In [26]:
dense.shape

(33563, 26260)

In [27]:
pos_label = [1 for b in pos_doc]
neg_label = [0 for b in neg_doc]
print(len(pos_label))
print(len(neg_label))

20887
5373


In [28]:
label = pos_label + neg_label

In [29]:
len(label)

26260

In [30]:
data_train_s, data_test_s, label_train_s, label_test_s = model_selection.train_test_split(dense.T, label, test_size=0.1)

In [31]:
estimator = RandomForestClassifier(verbose=True)

In [32]:
estimator.fit(data_train_s, label_train_s)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=True,
            warm_start=False)

In [33]:
estimator.score(data_test_s, label_test_s)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


0.8354912414318355

In [43]:
tuned_parameters = [{'n_estimators': [50, 70, 90, 110]}]#, 130, 150]}]#, 'max_features': ['auto', 'sqrt', 'log2', None]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=2, scoring='accuracy', n_jobs=-1)

In [44]:
clf.fit(data_train_s, label_train_s)

GridSearchCV(cv=2, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'n_estimators': [50, 70, 90, 110]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [45]:
print("==== グリッドサーチ")
print("  ベストパラメタ")
print(clf.best_estimator_)

==== グリッドサーチ
  ベストパラメタ
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [46]:
print("トレーニングデータでCVした時の平均スコア")
for params, mean_score, all_scores in clf.grid_scores_:
        print("{:.3f} (+/- {:.3f}) for {}".format(mean_score, all_scores.std() / 2, params))

トレーニングデータでCVした時の平均スコア
0.837 (+/- 0.001) for {'n_estimators': 50}
0.837 (+/- 0.001) for {'n_estimators': 70}
0.837 (+/- 0.001) for {'n_estimators': 90}
0.836 (+/- 0.001) for {'n_estimators': 110}


In [47]:
y_true, y_pred = label_test_s, clf.predict(data_test_s)
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.91      0.30      0.45       565
          1       0.84      0.99      0.91      2061

avg / total       0.85      0.84      0.81      2626

